# Данный ноутбук посвящен тестированию Бейзлайна


Разделы



## Инициализация

In [1]:
import os
import pathlib

ROOT_PATH = pathlib.Path(__name__).resolve().parent.parent
print(ROOT_PATH)
RANDOM_SEED = 42

/Users/andrey/PycharmProjects/vector-search-hse


In [2]:
os.chdir(ROOT_PATH)
os.getcwd()


'/Users/andrey/PycharmProjects/vector-search-hse'

## Загрузка локального поиска по видео

У нас есть поисковый индекс построенный на эмбеддингах фреймов с частотой в 1 секунду.

Метчинг к видео происходит с помощью метаданных к ним и отбора наиболее релевантных отрезков по видео

Для задачи поиска эмбеддингов используется линейная формула косинусного сходства (эмбединги нормализованы). Функция - `brute_force_torch`

In [3]:
import pickle

with open('index.pkl', 'rb') as handle:
    index = pickle.load(handle)

with open('metadata.pkl', 'rb') as handle:
    meta = pickle.load(handle)

with open('thumbnails.pkl', 'rb') as handle:
    thumbnails_meta = pickle.load(handle)

In [4]:
from typing import Any, List, Tuple, Dict
import clip
from numpy.typing import NDArray
import numpy as np

import torch


def brute_force_query_torch(X, x, certainty_threshold):
    sims = (x @ X.t()).squeeze(0)  # shape: [N]

    mask = sims >= certainty_threshold
    filtered_indices = torch.nonzero(mask).squeeze(1)  # индексы в X
    filtered_sims = sims[filtered_indices]

    # Сортировка по убыванию
    sorted_sims, order = torch.sort(filtered_sims, descending=True)
    sorted_indices = filtered_indices[order]

    return sorted_indices, sorted_sims.float()


from pydantic import BaseModel


class VideoDescription(BaseModel):
    name: str
    path: str
    video_id: int
    frame_num: int
    fps: int
    start_pos: float
    end_pos: float
    score: float


class UsedVideo(BaseModel):
    start_pos: float
    end_pos: float
    score: float


class ModelConfig(BaseModel):
    device: str = 'cpu'
    frame_threshold: float = 0.26
    percentile: float = 0.8
    video_threshold: float = 0.5


class LocalSearchEngine:

    def __init__(
        self,
        config: ModelConfig,
        index: List[NDArray],
        meta: List[Any],
        thumbnails_meta: Dict[str, Any],
    ) -> None:
        self.model, self.preprocessor = clip.load(
            'ViT-B/32',
            device=config.device,
        )
        self.dataset = torch.tensor(np.array(index))
        self.thumbnails_meta = thumbnails_meta
        self.meta = meta
        self.all_videos = sorted(set([m[0] for m in meta]))
        self.video_to_int = {v: i for i, v in enumerate(self.all_videos)}
        self.int_to_video = {i: v for v, i in self.video_to_int.items()}
        self.meta_video_ids = torch.tensor(
            [self.video_to_int[m[0]] for m in meta],
            device='cpu',
            dtype=torch.int32,
        )
        self.meta_frame_nums = torch.tensor(
            [m[1] for m in meta],
            device='cpu',
            dtype=torch.int32,
        )
        self.frame_threshold: float = config.frame_threshold
        self.percentile: float = config.percentile
        self.video_threshold: float = config.video_threshold

    def encode_text(
        self,
        text: str
    ) -> torch.Tensor:
        with torch.no_grad():
            data = self.model.encode_text(clip.tokenize([text]))
        data /= torch.linalg.norm(data)
        return data

    def encode_image(
        self,
        file: NDArray,
    ) -> torch.Tensor:
        with torch.no_grad():
            data = self.model.encode_image(self.preprocessor(file).unsqueeze(0))

        data = torch.sign(data) * torch.pow(torch.abs(data), 0.25)
        data /= torch.linalg.norm(data)
        return data

    def query_frames(
        self,
        x,
        threshold: float,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        indices, certs = brute_force_query_torch(self.dataset, x, threshold)
        certs = certs.cpu()
        return indices, certs

    def get_videos_by_text(
        self,
        text: str,
    ) -> List:
        x = self.encode_text(text)
        return self.query_videos_by_tensor(x)

    def query_videos_by_tensor(
        self,
        x: torch.Tensor,
        limit: int = 20
    ) -> List[VideoDescription]:
        indices, certs = brute_force_query_torch(self.dataset, x, self.frame_threshold)
        certs = certs.cpu()
        video_idxs = self.meta_video_ids[indices]
        video_frames = self.meta_frame_nums[indices]

        video_descriptions = []
        used_videos = {}

        vals, order = torch.sort(video_idxs)
        targets = torch.tensor(
            [self.video_to_int[v] for v in self.all_videos],
            device=video_idxs.device,
        )
        order = order.cpu()
        left = torch.bucketize(targets, vals, right=False).cpu()
        right = torch.bucketize(targets, vals, right=True).cpu()

        for i, video in enumerate(self.all_videos):
            if left[i] == right[i]:
                continue
            args = order[left[i]:right[i]]
            cert_ = certs[order[left[i] + int((right[i] - left[i]) * (1 - self.percentile))]]
            if cert_ < self.video_threshold:
                continue
            subset = video_frames[args]
            start_ = torch.min(subset)
            end_ = torch.max(subset)
            max_frame = subset[0]
            used_videos[video] = UsedVideo(start_pos=start_.item(), end_pos=end_.item(), score=cert_.item())

            video_description = VideoDescription(
                name=video.split('/')[-1],
                path=video,
                video_id=self.video_to_int[video],
                frame_num=max_frame,
                fps=self.thumbnails_meta[video][1],
                start_pos=start_.item(),
                end_pos=end_.item(),
                score=cert_.item()
            )
            video_descriptions.append(video_description)

        video_descriptions = sorted(video_descriptions, key=lambda x: x.score, reverse=True)[:limit]

        return video_descriptions

# Загрузка тегов

В предыдущем ноутбуке мы обрабатывали теги на видео, полученные с помощью разметчика.

Теперь преобразуем их в удобную форму для сравнения с поисковой выдачей из движка


In [17]:
import pandas as pd

tags_df = pd.read_parquet('data/tags.parquet')
tags_df.head()

,video_name,tag_ru,tag_en
0,IMG_0903.MOV,космос,space
1,IMG_0903.MOV,усы,mustache
2,IMG_0903.MOV,мужчина,man
3,IMG_0903.MOV,NASA,NASA
4,IMG_0718.MOV,актёр,actor


In [18]:
video_base = set(tags_df['video_name'].values)

test_queries = tags_df.groupby('tag_en')['video_name'].agg(list).reset_index()
test_queries['ttl_videos'] = test_queries['video_name'].apply(lambda x: len(x))

test_queries = test_queries[test_queries['ttl_videos'] > 10]

test_queries.sort_values(by='ttl_videos', ascending=False)


,tag_en,video_name,ttl_videos
164,man,"[IMG_0903.MOV, IMG_0718.MOV, IMG_1374.MOV, IMG...",145
288,woman,"[IMG_0724.MOV, IMG_1002.MOV, IMG_1567.MOV, IMG...",63
128,glasses,"[IMG_1374.MOV, IMG_0730.MOV, IMG_1177.MOV, IMG...",52
122,food,"[IMG_1363.MOV, IMG_0727.MOV, IMG_1834.MOV, IMG...",50
187,mustache,"[IMG_0903.MOV, IMG_1374.MOV, IMG_0730.MOV, IMG...",43
88,cooking,"[IMG_1834.MOV, IMG_1767.MOV, IMG_1561.MOV, IMG...",36
30,animal,"[IMG_1823.MOV, IMG_1016.MOV, IMG_0731.MOV, IMG...",35
48,beard,"[IMG_1177.MOV, IMG_0928.MOV, IMG_1834.MOV, IMG...",30
171,meme,"[IMG_1361.MOV, IMG_1363.MOV, IMG_0726.MOV, IMG...",28
127,game,"[IMG_0719.MOV, IMG_1564.MOV, IMG_1835.MOV, IMG...",26


In [19]:
query_dict = dict(zip(test_queries['tag_en'].values, test_queries['video_name'].values))


## Векторный поиск видео

Проинициализируем нашу модель и для каждого запроса из тега сформируем поисковую выдачу


In [29]:
config = ModelConfig(
    device='cpu',
    frame_threshold=0.2,
    video_threshold=0.2,
    percentile=0.8,
)
engine = LocalSearchEngine(
    config=config,
    index=index,
    meta=meta,
    thumbnails_meta=thumbnails_meta,
)



In [30]:
results_per_query = {}
tests_per_query = {}

for text_query, relevant_items in query_dict.items():
    result = engine.get_videos_by_text(text_query)
    results_per_query[text_query] = result
    # For ranking
    predicted_items = [video.name for video in result]
    tests_per_query[text_query] = {
        'relevant': relevant_items,
        'predicted': predicted_items,
    }

In [31]:
QUERY = 'horse'
print(f'Actual:', tests_per_query[QUERY]['relevant'][:10], '...')
print(f'Predicted:', tests_per_query[QUERY]['predicted'][:10], '...')

Actual: ['IMG_1823.MOV', 'IMG_1361.MOV', 'IMG_1361.MOV', 'IMG_1198.MOV', 'IMG_1198.MOV', 'IMG_1259.MOV', 'IMG_1259.MOV', 'IMG_1930.MOV', 'IMG_1930.MOV', 'IMG_1256.MOV'] ...
Predicted: ['IMG_1253.MOV', 'IMG_1256.MOV', 'IMG_1198.MOV', 'IMG_1823.MOV', 'IMG_1145.MOV', 'IMG_0818.MOV', 'IMG_0845.MOV', 'IMG_1361.MOV', 'IMG_1259.MOV', 'IMG_0955.MOV'] ...


## Оценка качества по запросам

Для оценки качества наших моделей применим ряд метрик:

- Precision $$P = \frac{P_n(relevant)}{K}$$
- Recal $$R = \frac{P_n(relevant)}{N_{relevant}}$$
- F1_score - гармоническое среднее между Precision и Recall.

 
- Метрика ранжирования **Average Precision at K**

$$ AP\@k = \sum_k (R_k - R_{k-1}) P_k $$ 


В тестах мы планируем показывать не всю выдачу, поэтому Recall и F1 будут выполнять вспомогательную роль для больших запросов и других тестов

Целевой метрикой по запросу для нас будет являться - **Average Precision at K** (подсчет кол-ва релевантных от 1 до K элементов в выдаче)


In [32]:
import numpy as np

# Реализация метрик

def precision_at_k(relevant_items: list[Any], predicted_ranked: list[Any], k=10):
    """Точность на первых k позициях"""
    if k > len(predicted_ranked):
        k = len(predicted_ranked)

    relevant_predicted = [item for item in predicted_ranked[:k] if item in relevant_items]
    return len(relevant_predicted) / k


def recall_at_k(relevant_items: list[Any], predicted_ranked: list[Any], k=10):
    """Полнота на первых k позициях"""
    if len(relevant_items) == 0:
        return 0.0

    relevant_predicted = [item for item in predicted_ranked[:k] if item in relevant_items]
    return len(relevant_predicted) / len(relevant_items)


def f1_score_at_k(relevant_items: list[Any], predicted: list[Any], k=10):
    precision = precision_at_k(relevant_items, predicted, k)
    recall = recall_at_k(relevant_items, predicted, k)
    return 2 * (precision * recall) / (
        recall + recall + 1e-10
    )

def apk(
    actual: list[Any],
    predicted: list[Any],
    k: int = 10,
) -> float:
    """Метрика Average Precision at K"""
    if not actual:
        return 0.0

    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    return score / min(len(actual), k)


In [33]:
# Утилиты для подсчета метрик
def eval_query_metrics(
    relevant: list[str],
    predicted: list[str],
    k: int = 10,
) -> float:
    results = {}

    # Бинарные метрики
    results['total_relevant'] = len(relevant)
    results['total_predicted'] = len(predicted)
    results[f'precision@{k}'] = precision_at_k(relevant, predicted, k)
    results[f'recall@{k}'] = recall_at_k(relevant, predicted, k)
    results[f'f1_score@{k}'] = f1_score_at_k(relevant, predicted, k)
    results[f'average_precision@{k}'] = apk(relevant, predicted, k)

    return results

def calc_for_queries(
    ranked_results: Dict[str, Dict[str, List[Any]]],
    k: int = 10,
) -> Dict[str, Any]:
    
    metrics = {}
    
    for query, result in ranked_results.items():
        metrics_by_query = eval_query_metrics(result['relevant'], result['predicted'], k)
        metrics[query] = metrics_by_query
    
    return metrics


In [34]:
K_TOP = 10

def get_metrics_for_queries(k: int = 10) -> Dict[str, Any]:
    metrics_per_query = calc_for_queries(
        tests_per_query,
        k=k,
    )
    
    stat_df = pd.DataFrame.from_dict(metrics_per_query, orient='index')
    stat_df.index.name = 'query'
    stat_df.reset_index(inplace=True)
    print('Results')
    return metrics_per_query, stat_df

top_10_metrics, top_10_metrics_df = get_metrics_for_queries(K_TOP)

display(top_10_metrics_df.sort_values(by=f'average_precision@{K_TOP}', ascending=False))

Results


,query,total_relevant,total_predicted,precision@10,recall@10,f1_score@10,average_precision@10
23,woman,63,20,0.9,0.142857,0.9,0.900000
14,horse,19,20,0.9,0.473684,0.9,0.900000
8,dog,19,20,0.8,0.421053,0.8,0.753214
12,glasses,52,20,0.7,0.134615,0.7,0.565873
16,man,145,20,0.7,0.048276,0.7,0.545873
3,car,22,20,0.6,0.272727,0.6,0.529167
20,musical instrument,15,20,0.6,0.400000,0.6,0.509167
21,mustache,43,20,0.6,0.139535,0.6,0.440556
2,beard,30,20,0.6,0.200000,0.6,0.432698
9,drawing,21,20,0.5,0.238095,0.5,0.377222


In [35]:
print(top_10_metrics_df.sort_values(by=f'average_precision@{K_TOP}', ascending=False).to_markdown())

|    | query              |   total_relevant |   total_predicted |   precision@10 |   recall@10 |   f1_score@10 |   average_precision@10 |
|---:|:-------------------|-----------------:|------------------:|---------------:|------------:|--------------:|-----------------------:|
| 23 | woman              |               63 |                20 |            0.9 |   0.142857  |           0.9 |              0.9       |
| 14 | horse              |               19 |                20 |            0.9 |   0.473684  |           0.9 |              0.9       |
|  8 | dog                |               19 |                20 |            0.8 |   0.421053  |           0.8 |              0.753214  |
| 12 | glasses            |               52 |                20 |            0.7 |   0.134615  |           0.7 |              0.565873  |
| 16 | man                |              145 |                20 |            0.7 |   0.0482759 |           0.7 |              0.545873  |
|  3 | car                |

## Общая метрика качества

Для подсчета общей метрики качества предлагаем воспользоваться стандартной метрикой ранжирования Mean Average Precision@K

$$ MAP\@K = \frac{1}{K} \sum_{n=1}^{K} AP\@n $$

In [36]:
def mapk(
    actual: list[list[Any]],
    predicted: list[list[Any]],
    k=10,
):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

In [37]:
relevant_plain = [val['relevant'] for val in tests_per_query.values()]
predicted_plain = [val['predicted'] for val in tests_per_query.values()]
mean_apk = mapk(relevant_plain, predicted_plain, k=10)
print(f'Mean Average Precision at {K_TOP}:', mean_apk)

Mean Average Precision at 10: 0.332349537037037


## Промежуточные результаты


Мы получили метрики бейзлайна с базовыми параметрами на основании нашего пайплайна Text-To-Image-To-Video

Для части запросов мы получили достаточно хороший результат в по AP@K.

Запросы dog, horse, car - дали нам выдачу больше 70% по average precision по 10 объектам.

Однако есть запросы, в которых существует совсем плохие метрики отбора - meme, People, plate


### Заметки

- Проверить качество разметки и возможно исключить нерелевантные теги
- Провести валидацию и подбор threshold 